In [1]:
import sqlite3
import pandas as pd

<!-- 
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+
 -->

In [11]:
# +--------------+---------+
# | Column Name  | Type    |
# +--------------+---------+
# | id           | int     |
# | name         | varchar |
# | salary       | int     |
# | departmentId | int     |
# +--------------+---------+

# +-------------+---------+
# | Column Name | Type    |
# +-------------+---------+
# | id          | int     |
# | name        | varchar |
# +-------------+---------+ 

In [12]:
# create an in-memory database
conn = sqlite3.connect(':memory:')

# create a table and insert some data
conn.execute('''CREATE TABLE employee
             (id INT   
			 ,name varchar   
			 ,salary DECIMAL(10, 2)    
			 ,departmentId INT)
			  ''')

# create a table and insert some data
conn.execute('''CREATE TABLE department
             (id INT   
			 ,name varchar)
			  ''')

conn.execute("INSERT INTO employee VALUES (1, 'Randa', 70000, 1)")
conn.execute("INSERT INTO employee VALUES (2, 'Adel', 50000, 3)")
conn.execute("INSERT INTO employee VALUES (3, 'Naguib', 55000, 2)")
conn.execute("INSERT INTO employee VALUES (4, 'John', 80000, 1)")
conn.execute("INSERT INTO employee VALUES (5, 'Smith', 99000, 3)")
conn.execute("INSERT INTO employee VALUES (6, 'Mike', 120000, 2)")
conn.execute("INSERT INTO employee VALUES (7, 'Wiliam', 85000, 1)")
conn.execute("INSERT INTO employee VALUES (7, 'Ola', 85000, 1)")
conn.execute("INSERT INTO employee VALUES (8, 'Mike', 50000, 2)")
conn.execute("INSERT INTO employee VALUES (9, 'Wiliam', 80000, 3)")
conn.execute("INSERT INTO employee VALUES (10, 'Mike', 181000, 2)")
conn.execute("INSERT INTO employee VALUES (11, 'Alice', 123000, 2)")
conn.execute("INSERT INTO employee VALUES (12, 'Norman', 330000, 3)")


conn.execute("INSERT INTO department VALUES (1, 'IT')")
conn.execute("INSERT INTO department VALUES (2, 'Marketing')")
conn.execute("INSERT INTO department VALUES (3, 'Sales')")


pd.read_sql_query("SELECT * from Employee", conn)
# conn.close()

,id,name,salary,departmentId
0,1,Randa,70000,1
1,2,Adel,50000,3
2,3,Naguib,55000,2
3,4,John,80000,1
4,5,Smith,99000,3
5,6,Mike,120000,2
6,7,Wiliam,85000,1
7,7,Ola,85000,1
8,8,Mike,50000,2
9,9,Wiliam,80000,3


In [13]:
pd.read_sql_query('''

    SELECT Department, Employee, Salary 
    FROM
    (
        SELECT d.name AS Department, 
        e.name AS Employee, 
        e.salary AS Salary,
        DENSE_RANK() OVER(PARTITION BY e.departmentId ORDER BY e.salary DESC) AS rank_salary
        FROM 
        Employee e 
        JOIN Department d ON e.departmentId = d.id
    )sub
    Where rank_salary<=2

                  ''', conn)

,Department,Employee,Salary
0,IT,Wiliam,85000
1,IT,Ola,85000
2,IT,John,80000
3,Marketing,Mike,181000
4,Marketing,Alice,123000
5,Sales,Norman,330000
6,Sales,Smith,99000


In [14]:
pd.read_sql_query('''

   SELECT *
   FROM Employee ORDER BY salary DESC

                  ''', conn)

,id,name,salary,departmentId
0,12,Norman,330000,3
1,10,Mike,181000,2
2,11,Alice,123000,2
3,6,Mike,120000,2
4,5,Smith,99000,3
5,7,Wiliam,85000,1
6,7,Ola,85000,1
7,4,John,80000,1
8,9,Wiliam,80000,3
9,1,Randa,70000,1


In [15]:
pd.read_sql_query('''

      SELECT DISTINCT(salary) 
      from Employee 
      order by salary DESC
      LIMIT 1 OFFSET 1

                  ''', conn)

,salary
0,181000


In [16]:
pd.read_sql_query('''

    SELECT CAST(salary AS varchar) AS sales_varchar
    FROM Employee

                  ''', conn)

,sales_varchar
0,70000
1,50000
2,55000
3,80000
4,99000
5,120000
6,85000
7,85000
8,50000
9,80000


In [19]:
pd.read_sql_query('''
    SELECT
    salary,
    (ROW_NUMBER() OVER(ORDER BY salary) -1) % 5 AS mod_5
    FROM
    Employee
                  ''', conn)

,salary,mod_5
0,50000,0
1,50000,1
2,55000,2
3,70000,3
4,80000,4
5,80000,0
6,85000,1
7,85000,2
8,99000,3
9,120000,4
